<a href="https://colab.research.google.com/github/cardstdani/practica-par/blob/main/PracticaParadigmas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Práctica Paradigmas 1**

In [ ]:
!pip install pyvis==0.3.1

In [26]:
import random
import pandas as pd
from IPython.display import display, HTML

class TrieNode:
    def __init__(self, inputChar):
        self.char = inputChar
        self.end = False
        self.children = {}
 
class Trie():
    def __init__(self, startingElements=None):
        self.root = TrieNode("")
        if startingElements!=None:
          for i in startingElements: self.insert(i)
    def insert(self, word):
        node = self.root
        for char in word:
            if char in node.children:
                node = node.children[char]
            else:
                new_node = TrieNode(char)
                node.children[char] = new_node
                node = new_node
        node.end = True      
    def searchAndSplit(self, x):
        node = self.root
        output = ["", ""]
        for char in x:
            if char in node.children:
                node = node.children[char]            
            else:
                return []        
            output[1 if node.end else 0] += node.char
        return output if node.end else []
    def toGraph(self):
      from pyvis.network import Network
      g = Network(directed =True)      
      g.show_buttons()

      nodeIndex = 1
      currentNode = 0
      q = [self.root]      
      g.add_node(currentNode, label="", color="red")
      tempLabels = {0:""}
      while q!=[]:
        n = q.pop(0)                     
        for i in n.children.values():
          tempLabels[nodeIndex] = tempLabels[currentNode]+i.char
          g.add_node(nodeIndex, label=tempLabels[currentNode]+i.char, color="#48e073" if i.end else "blue")
          g.add_edge(currentNode, nodeIndex)
          nodeIndex+=1
          q.append(i)
        currentNode+=1
      g.show('nx.html')

class MainGame:
  def __init__(self):
    self.matrix = [random.sample(["."]*45+["a"]*18+["b"]*4+["c"]*3+["1"]*2, 6) for i in range(6)]
    self.objects = {".":[".",0],"a":["b",1],"b":["c",5],"c":["d",25],"d":["e",125],"e":["e",625],"1":["1",-25],"2":["3",-5],"3":["4",50],"4":["4",500],"x":["x",-50]}
    self.turn = 0
    self.storage = "."
    self.bigFoots = [[[i,j], 0] for i in range(len(self.matrix)) for j in range(len(self.matrix[0])) if self.matrix[i][j]=="1"]
    self.actual = random.choice(["a"]*30+["b"]*5+["c"]*1+["1"]*6+["w"]*1)
    self.tr = Trie((''.join(chr(97+int(j)) for j in str(i))+str(k) for k in range(len(self.matrix[0])) for i in range(len(self.matrix))))
    self.tr.insert("exit")
    self.tr.insert("*")
    self.tr.toGraph()

  def main(self):
    print("Que empiece el juego:\U0001F609")
    self.showGame()    
    while any("." in x for x in self.matrix):
      message = self.validarEntrada(input("Mover a casilla: ").lower().replace(" ", ""))
      while not message[0]: message = self.validarEntrada(input("Jugada errónea\nMover a casilla: ").lower().replace(" ", ""))

      if "".join(message[1])=="exit": break
      if "".join(message[1])=="*": self.storage=self.actual; self.showGame(); continue
      coordinates = (lambda l: [int("".join([str(ord(i)%97) for i in l[0]])), int(l[1])])(message[1])
      if self.matrix[coordinates[0]][coordinates[1]]!="." or (self.actual=="w" and self.matrix[coordinates[0]][coordinates[1]]=="."): print("Jugada errónea"); continue

      self.updateMatrix(coordinates)
      self.updateActual()
      self.turn+=1
      self.bigFoots = [[i[0], i[1]+1] for i in self.bigFoots]
      self.showGame()
    print("Partida terminada, GG:\U0001F44F")
  
  def updateMatrix(self, coordinates):
    if self.actual=="w": self.matrix[coordinates[0]][coordinates[1]] = "."; return    
    self.matrix[coordinates[0]][coordinates[1]] = self.actual
    if self.actual=="1": self.updateBigFoots(); self.bigFoots.append([coordinates, 0]); return;

    g = self.getGroup(coordinates)
    while len(g)>2:
      for i in g: self.matrix[i[0]][i[1]] = "."
      self.matrix[coordinates[0]][coordinates[1]] = self.objects[self.actual][0]
      g = self.getGroup(coordinates)
    self.updateBigFoots()
  
  def updateBigFoots(self):
    for i in range(len(self.bigFoots)):
      n = self.bigFoots[i][:]
      for j in (lambda l:[k for k in l if k[0]>=0 and k[1]>=0])([[n[0][0]-1, n[0][1]], [n[0][0], n[0][1]+1], [n[0][0]+1, n[0][1]], [n[0][0], n[0][1]-1]]):
        try:
          if self.matrix[j[0]][j[1]]==".": self.matrix[j[0]][j[1]]="1"; self.matrix[n[0][0]][n[0][1]]="x" if n[1]>10 else "."; self.bigFoots[i][0]=j[:]; break;
        except:
          pass

      if n[0]==self.bigFoots[i][0]: 
        g = self.getGroup(n[0], True)
        if not "." in g:
          if len(g)>2:
            deletedObjs = []
            for j in g: self.matrix[j[0]][j[1]] = "."; bfIndex = [k for k in range(len(self.bigFoots)) if self.bigFoots[k][0]==j][0]; deletedObjs.append(self.bigFoots.pop(bfIndex));
            deletedObjs = max(deletedObjs, key=lambda x:x[1])[0]
            self.matrix[deletedObjs[0]][deletedObjs[1]] = "3"
          else:
            for j in g: self.matrix[j[0]][j[1]] = "2"; self.bigFoots.pop([k for k in range(len(self.bigFoots)) if self.bigFoots[k][0]==j][0]);

  def getGroup(self, coordinates, bigFootMode=False):    
    self.visitedMatrix = [[False]*len(self.matrix[0]) for i in range(len(self.matrix))]
    output = [coordinates]
    q = [coordinates]
    while q!=[]:
      n = q.pop(0)
      self.visitedMatrix[n[0]][n[1]] = True
      for i in (lambda l:[j for j in l if j[0]>=0 and j[1]>=0])([[n[0]-1, n[1]], [n[0], n[1]+1], [n[0]+1, n[1]], [n[0], n[1]-1]]):
        try:
          if (not self.visitedMatrix[i[0]][i[1]]) and (self.matrix[i[0]][i[1]]==self.matrix[coordinates[0]][coordinates[1]] or (bigFootMode and self.matrix[i[0]][i[1]]==".")): q.append(i); output.append(i);
        except:
          pass
    return output

  def updateActual(self):
    self.actual = random.choice(["a"]*30+["b"]*5+["c"]*1+["1"]*6+["w"]*1)
  
  def validarEntrada(self, s):
    s = self.tr.searchAndSplit(s)
    return (True if s else False, s)

  def showGame(self):
    df = pd.DataFrame(self.matrix, columns=list(range(len(self.matrix[0]))), index=[''.join(chr(65+int(j)) for j in str(i)) for i in range(len(self.matrix))])
    display(HTML(df.to_html()))
    print(f"\nTurno: {self.turn} Puntos:{sum(self.objects[j][1] for i in self.matrix for j in i)}\nAlmacen: [{self.storage}] Actual: [{self.actual}]")

if __name__=="__main__":
  g = MainGame()
  g.main()

Que empiece el juego:😉


,0,1,2,3,4,5
A,.,.,.,.,a,.
B,.,.,a,.,1,.
C,.,.,.,.,a,.
D,.,b,a,.,.,.
E,a,.,.,.,.,.
F,a,.,.,a,.,a



Turno: 0 Puntos:-12
Almacen: [.] Actual: [a]
Mover a casilla: c5


,0,1,2,3,4,5
A,.,.,.,.,a,.
B,.,.,a,.,.,1
C,.,.,.,.,a,a
D,.,b,a,.,.,.
E,a,.,.,.,.,.
F,a,.,.,a,.,a



Turno: 1 Puntos:-11
Almacen: [.] Actual: [a]
Mover a casilla: b4


...

,0,1,2,3,4,5
A,.,.,.,a,.,2
B,a,.,1,b,1,.
C,.,.,.,.,a,.
D,.,b,a,.,.,.
E,a,.,.,.,.,.
F,a,.,.,a,.,a



Turno: 11 Puntos:-37
Almacen: [.] Actual: [a]
Mover a casilla: a5
Jugada errónea
Mover a casilla: b5


,0,1,2,3,4,5
A,.,.,1,a,1,2
B,a,.,.,b,.,a
C,.,.,.,.,a,.
D,.,b,a,.,.,.
E,a,.,.,.,.,.
F,a,.,.,a,.,a



Turno: 12 Puntos:-36
Almacen: [.] Actual: [a]
Mover a casilla: b4


IndexError: ignored